In [1]:
import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage


In [2]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [3]:
from galileo.handlers.langchain import GalileoCallback
from galileo import GalileoDecorator

os.environ["GALILEO_PROJECT"] = "sessions-demo"
os.environ["GALILEO_LOG_STREAM"]="dev"

os.environ["LANGCHAIN_PROJECT"]="galileo-sessions-demo"

# Create a callback with the custom logger
galileo_v2_callback = GalileoCallback(
    start_new_trace=True,   # Whether to start a new trace for each chain
    flush_on_chain_end=True # Whether to flush traces when chains end
) 
galileo_context = GalileoDecorator()

from braintrust import init_logger
from braintrust_langchain import BraintrustCallbackHandler

init_logger(project="Sessions Demo", api_key=os.environ.get("BRAINTRUST_API_KEY"))
braintrust_callback = BraintrustCallbackHandler()

In [ ]:
def create_chat_session():
    # Initialize the language model
    llm = ChatOpenAI(model="gpt-3.5-turbo", callbacks=[galileo_v2_callback, braintrust_callback])
    
    # Create a prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant who responds in a friendly and concise manner."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])

    # Create the chain
    chain = (
        {"input": lambda x: x["input"], "history": lambda x: x["history"]}
        | prompt
        | llm
        | StrOutputParser()
    )
    galileo_context.start_session(name="State capitals")
    return chain

def terminate_chat_session():
    print("\nAssistant: Goodbye! Have a great day!")
    # galileo_context.flush()

def main():
    print("Welcome to Simple LangChain Chat!")
    print("Type 'exit' to end the conversation.")
    print("-" * 50)
    
    # Create the chat chain
    chat_chain = create_chat_session()
    
    # Initialize conversation history
    conversation_history = []
    
    while True:
        # Get user input
        user_input = input("\nYou: ")
        
        # Check if user wants to exit
        if user_input.lower() in ["exit", "quit", "bye"]:
            terminate_chat_session()
            break
        
        try:
            # Process the user input through the chain
            response = chat_chain.invoke({
                "input": user_input,
                "history": conversation_history
            })

            # Update conversation history
            conversation_history.append(HumanMessage(content=user_input))
            conversation_history.append(AIMessage(content=response))
            
            # Display the response
            print(f"\nAssistant: {response}")
        except Exception as e:
            print(f"\nError: {str(e)}")


In [5]:
main()

Welcome to Simple LangChain Chat!
Type 'exit' to end the conversation.
--------------------------------------------------

Assistant: The capital of California is Sacramento.

Assistant: San Francisco is not the capital of California because Sacramento was chosen as the state's capital in 1854 due to its central location and its proximity to the mining regions of the Sierra Nevada.

Assistant: The capital of New York is Albany.

Assistant: Albany was chosen as the capital of New York over New York City in 1797 due to its central location within the state and to prevent any one city from having too much power over the government.

Assistant: Goodbye! Have a great day!
